This file reproduce the plots present of the paper on MNIST dataset based on the data produced by running reproduce_mnist0.py and reproduce_mnist1.py.

The parameters on each of the two experiments must correspond to the parameter of reproduce_mnist0.py (for the first part) and reproduce_mnist1.py (for the second part). 

## Analysis of experiments on MNIST dataset and Two Worlds graph

In [ ]:

import study
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

result_directory = "results-data/"
plot_directory = "results-plot/"


# Base parameters for the MNIST experiments
params_mnist = {
  "batch-size": 64,
  "model": "simples-minimalcnn",
  "loss": "nll",
  "evaluation-delta": 10,
  "subsampled-evaluation":2,
  "nb-steps": 200,
  "momentum-at": "worker",
  "momentum":0.9,
  "dampening":0.9,
  "learning-rate": 0.1,
  "learning-rate-decay-delta": 40,
  "learning-rate-decay": 40,
  "dataset": "mnist",
  "numb-labels": 10,
  "nb-honests":26,
  "topology-name" : "two_worlds",
  "topology-hyper": 8,
  "topology-weights": "metropolis",
  "dirichlet-alpha":1
  }

# Hyperparameters to test
gars = ["CSplus_RG", "GTS_RG", "CShe_RG", 'IOS']
attacks = [ "spectral", "sparse_little", "dissensus" , "sparse_empire"]#, "signflipping", "labelflipping"]
dataset = "mnist"
byzcounts = [1,2,3,4,5,6,7,8,9,10,11]

alpha = params_mnist["dirichlet-alpha"]
alphas = [alpha]
momentums = [params_mnist['momentum']]
params_mnist["dampening"] = params_mnist["momentum"]

params = params_mnist

# Jobs
seeds=tuple(range(1))


algorithms_name = {
          "CSplus_RG":r"CS$_{+}$-RG",
          "GTS_RG":r"GTS-RG",
          "IOS":r"IOS",
          "CShe_RG":r"ClippedGossip",
          "dsgd":r"D-SGD",
          "cva":r"$\rm GTS$-$\rm RG$",
          "trmean":"BRIDGE",
          "rfa":"MoGM",
          "centeredclip":r"$\rm CS_{He}$-$\rm RG$",
          "cgplus":r"$CS$_{ours}$-RG$"
        }

attack_name = {
          "dissensus":"Dissensus", 
          "spectral":"SpH" , 
          "sparse_empire":"FOE", 
          "sparse_little":"ALIE"
        }


In [ ]:
def aggregate_experiment(name_experiment, seeds=seeds, result_directory=result_directory):
    """
    aggregate values of the experiments, output the min, max, mean, std of each experiment batch.
    """
    df_list = [pd.read_csv(result_directory + "/" + name_experiment + "-" +str(seed) + "/eval", delimiter='\t') for seed in seeds]

    stacked_df = np.stack([df[['Cross-accuracy', 'Average loss']].values for df in df_list], axis=2)

    # Calculate mean, std, min, and max across the third dimension (the DataFrame axis)
    mean_acc = stacked_df[:, 0, :].mean(axis=1)
    std_acc = stacked_df[:, 0, :].std(axis=1)
    min_acc = stacked_df[:, 0, :].min(axis=1)
    max_acc = stacked_df[:, 0, :].max(axis=1)

    mean_loss = stacked_df[:, 1, :].mean(axis=1)
    std_loss = stacked_df[:, 1, :].std(axis=1)
    min_loss = stacked_df[:, 1, :].min(axis=1)
    max_loss = stacked_df[:, 1, :].max(axis=1)

    # Create a new DataFrame to store the results, with "# Step number" as index
    result_df = pd.DataFrame({
        'mean-acc': mean_acc,
        'std-acc': std_acc,
        'min-acc': min_acc,
        'max-acc': max_acc,
        'mean-loss': mean_loss,
        'std-loss': std_loss,
        'min-loss': min_loss,
        'max-loss': max_loss
    }, index=df_list[0]["# Step number"])

    return result_df

In [ ]:
### DEPRECATED
def compute_avg_err_op(name, location, *colops, avgs="", errs="-err"):
  """ Compute the average and standard deviation of the selected columns over the given experiment.
  Args:
    name Given experiment name
    location Script to read from
    ...  Tuples of (selected column name (through 'study.select'), optional reduction operation name)
    avgs Suffix for average column names
    errs Suffix for standard deviation (or "error") column names
  Returns:
    Data frames for each of the computed columns,
    Tuple of reduced values per seed (or None if None was provided for 'op')
  Raises:
    'RuntimeError' if a reduction operation was specified for a column selector that did not select exactly 1 column
  """
# Load all the runs for the given experiment name, and keep only a subset
  datas = tuple(study.select(study.Session(result_directory + "/" + name + "-" +str(seed), location), *(col for col, _ in colops)) for seed in seeds)

  # Make the aggregated data frames
  def make_df_ro(col, op):
    nonlocal datas
    # For every selected columns
    subds = tuple(study.select(data, col) for data in datas)
    df    = pd.DataFrame(index=subds[0].index)
    ro    = None
    for cn in subds[0]:
      # Generate compound column names
      avgn = cn + avgs
      errn = cn + errs
      # Compute compound columns
      numds = np.stack(tuple(subd[cn].to_numpy() for subd in subds))
      df[avgn] = numds.mean(axis=0)
      df[errn] = numds.std(axis=0)
      # Compute reduction, if requested
      if op is not None:
        if ro is not None:
          raise RuntimeError(f"column selector {col!r} selected more than one column ({(', ').join(subds[0].columns)}) while a reduction operation was requested")
        ro = tuple(getattr(subd[cn], op)().item() for subd in subds)
    # Return the built data frame and optional computed reduction
    return df, ro
  dfs = list()
  ros = list()
  for col, op in colops:
    df, ro = make_df_ro(col, op)
    #df = df.replace(np.nan, np.inf)
    #df.dropna()
    dfs.append(df)
    ros.append(ro)
  # Return the built data frames and optional computed reductions
  return dfs, ros

### Ploting the accuracy as a function of the number of Byzantin neighbors.

In [ ]:
size=24
size_legend=24
mpl.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.serif': 'Roman',
    'font.weight':'bold',
    #'legend.fontweight':'bold',
    'text.usetex': True,
    'pgf.rcfonts': False,
    "axes.grid" : True,
    'font.size': size,
    'axes.labelsize':size,
    'axes.titlesize':size,
    'figure.titlesize':size,
    'xtick.labelsize':size,
    'ytick.labelsize':size,
    'legend.fontsize':size_legend
})
palette = sns.color_palette('colorblind') # 
colors = colors = plt.rcParams['axes.prop_cycle'].by_key()['color']# [palette[i] for i in range(5)] # ['#383F51','#B0413E','#FEA82F','#43AA8B','#6C7D47']# 
markers = ['o', 'v', 's', '*', 'd']
dashstyle = ['-', '--', '-.', ':', ':']
c=[]
m=[]

fig, axs = plt.subplots(1, len(attacks), figsize=(20, 5), sharex=False, sharey=True)

alpha = alphas[0]

#D-SGD
name_experiment_dsgd = f"{dataset}-average-h_{params['nb-honests']}-{params["topology-name"]}_{params["topology-hyper"]}-m_{momentum}-alpha_{alpha}_dsgd"

for attack_id, attack in enumerate(attacks):
    for momentum_id, momentum in enumerate(momentums):
        for gar_id, gar in enumerate(gars):
            X=[]
            Y=[]
            Yerr=[]
            Ymin=[]
            Ymax=[]
            for f in byzcounts:
                #Gar generic
                name_experiment = f"{dataset}-{attack}-{gar}-f_{f}-h_{params['nb-honests']}-{params["topology-name"]}_{params["topology-hyper"]}-m_{momentum}-alpha_{alpha}"
                #cols, _ = compute_avg_err_op(name_experiment, "eval",())#, ("Accuracy", "max"))
                df_expe = aggregate_experiment(name_experiment)
                #print(df_expe.info())
                #cols[0].replace(np.nan, 100)
                X.append(f)
                #print(cols[0].columns)
                y = df_expe.loc[200, "mean-acc"] #cols[0].loc[190:200, "Cross-accuracy"].mean()
                if np.isnan(y):
                    Y.append(1000)
                else:
                    Y.append(y)
                
                Yerr.append(df_expe.loc[200, "std-acc"])
                Ymin.append(df_expe.loc[200, "min-acc"])
                Ymax.append(df_expe.loc[200, "max-acc"])
            
            Y=np.array(Y)
            Yerr=np.array(Yerr)

            axs[attack_id].plot(X,Y, color=colors[gar_id], marker=markers[gar_id], label=algorithms_name[gar], linestyle=dashstyle[gar_id])
            axs[attack_id].fill_between(X, Ymin, Ymax, facecolor=colors[gar_id], alpha=0.2)
    
    axs[attack_id].set_title(attack_name[attack])
    axs[attack_id].set_xlabel(r"b")
    axs[attack_id].set_xticks(ticks=range(2,11,2))

axs[0].set_ylabel("accuracy",rotation=90,size="large")
#axs[0].set_yscale("log")
axs[0].set_ylim(bottom=0, top= 1)


handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.52, 0.05), ncol=5, labelspacing=0.1, handletextpad=0.1, borderaxespad=0)
fig.tight_layout(w_pad=0.2)


name_plot = f"acc_{dataset}-f_{byzcounts[0]}:{byzcounts[-1]}-h_{params['nb-honests']}-{params["topology-name"]}_{params["topology-hyper"]}-m_{momentum}-alpha_{alpha}"
fig.savefig(plot_directory + name_plot+'.pdf',bbox_inches='tight')

### Ploting the loss as a function of the number of Byzantin neighbors.

In [ ]:
size=24
size_legend=24
mpl.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.serif': 'Roman',
    'font.weight':'bold',
    #'legend.fontweight':'bold',
    'text.usetex': True,
    'pgf.rcfonts': False,
    "axes.grid" : True,
    'font.size': size,
    'axes.labelsize':size,
    'axes.titlesize':size,
    'figure.titlesize':size,
    'xtick.labelsize':size,
    'ytick.labelsize':size,
    'legend.fontsize':size_legend
})
palette = sns.color_palette('colorblind') # 
colors = colors = plt.rcParams['axes.prop_cycle'].by_key()['color']# [palette[i] for i in range(5)] # ['#383F51','#B0413E','#FEA82F','#43AA8B','#6C7D47']# 
markers = ['o', 'v', 's', '*', 'd']
dashstyle = ['-', '--', '-.', ':', ':']
c=[]
m=[]

fig, axs = plt.subplots(1, len(attacks), figsize=(20, 5), sharex=False, sharey=True)


alpha = alphas[0]

#D-SGD
name_experiment_dsgd = f"{dataset}-average-h_{params['nb-honests']}-{params["topology-name"]}_{params["topology-hyper"]}-m_{momentum}-alpha_{alpha}_dsgd"

for attack_id, attack in enumerate(attacks):
    for momentum_id, momentum in enumerate(momentums):
        for gar_id, gar in enumerate(gars):
            X=[]
            Y=[]
            Yerr=[]
            Ymin=[]
            Ymax=[]
            for f in byzcounts:
                #Gar generic
                name_experiment = f"{dataset}-{attack}-{gar}-f_{f}-h_{params['nb-honests']}-{params["topology-name"]}_{params["topology-hyper"]}-m_{momentum}-alpha_{alpha}"
                #cols, _ = compute_avg_err_op(name_experiment, "eval",())#, ("Accuracy", "max"))
                df_expe = aggregate_experiment(name_experiment)
                # print(df_expe.info())
                #cols[0].replace(np.nan, 100)
                X.append(f)
                #print(cols[0].columns)
                y = df_expe.loc[200, "mean-loss"] #cols[0].loc[190:200, "Cross-accuracy"].mean()
                if np.isnan(y):
                    Y.append(1e6)
                else:
                    Y.append(y)
                
                Yerr.append(df_expe.loc[200, "std-loss"])
                Ymin.append(df_expe.loc[200, "min-loss"])
                Ymax.append(df_expe.loc[200, "max-loss"])
            
            Y=np.array(Y)
            Yerr=np.array(Yerr)

            axs[attack_id].plot(X,Y, color=colors[gar_id], marker=markers[gar_id], label=algorithms_name[gar], linestyle=dashstyle[gar_id])
            axs[attack_id].fill_between(X, Ymin, Ymax, facecolor=colors[gar_id], alpha=0.2)

    axs[attack_id].set_title(attack_name[attack])
    axs[attack_id].set_xlabel(r"b")
    axs[attack_id].set_xticks(ticks=range(2,11,2))
axs[0].set_ylabel("train loss",rotation=90,size="large")
axs[0].set_yscale("log")
axs[0].set_ylim(bottom=0.5e-1, top= 2e2)


handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.52, 0.05), ncol=5, labelspacing=0.1, handletextpad=0.1, borderaxespad=0)
fig.tight_layout(w_pad=0.2)


name_plot = f"loss_{dataset}-f_{byzcounts[0]}:{byzcounts[-1]}-n_{params['nb-honests']}-{params["topology-name"]}_{params["topology-hyper"]}-m_{momentum}-alpha_{alpha}"
fig.savefig(plot_directory + name_plot+'.pdf',bbox_inches='tight')

# Analysis of error = f(connectivity), based on Erdos Renyi experiment

In [ ]:
### Import configuration
# Base parameters for the MNIST experiments
params = {
  "batch-size": 64,
  "model": "simples-minimalcnn",
  "loss": "nll",
  # "l2-regularize": 1e-4,
  "evaluation-delta": 10,
  "subsampled-evaluation":2,
  "nb-steps": 200,
  "momentum-at": "worker",
  "momentum":0.9,
  "dampening":0.9,
  "learning-rate": 0.1,
  "learning-rate-decay-delta": 40,
  "learning-rate-decay": 40,
  "dataset": "mnist",
  "numb-labels": 10,
  "nb-honests":20,
  "topology-name" : "Erdos_Renyi",
  "topology-weights": "metropolis",
  "dirichlet-alpha":1
  }
  

# Hyperparameters to test
gars = ["CSplus_RG", "GTS_RG", "CShe_RG", 'IOS']
attacks = ["dissensus", "spectral" ,"sparse_little", "sparse_empire"]#, "signflipping", "labelflipping"]
dataset = params["dataset"]
byzcounts = [4]
topology_hypers = list(np.linspace(0.26,1,11))


alpha = params["dirichlet-alpha"]
momentum = params['momentum']
params_mnist["dampening"] = params_mnist["momentum"]

f = byzcounts[0]
params_common = params
alphas = [alpha] # heterogeneity - the lower the more heterogeneous [1,5]
seeds=range(1)


font_size=24
font_size_legend=24
mpl.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.serif': 'Roman',
    'font.weight':'bold',
    #'legend.fontweight':'bold',
    'text.usetex': True,
    'pgf.rcfonts': False,
    "axes.grid" : True,
    'font.size': font_size,
    'axes.labelsize':font_size,
    'axes.titlesize':font_size,
    'figure.titlesize':font_size,
    'xtick.labelsize':font_size,
    'ytick.labelsize':font_size,
    'legend.fontsize':font_size_legend
})
palette = sns.color_palette('colorblind') # 
colors = colors = plt.rcParams['axes.prop_cycle'].by_key()['color']# [palette[i] for i in range(5)] # ['#383F51','#B0413E','#FEA82F','#43AA8B','#6C7D47']# 
markers = ['o', 'v', 's', '*', 'd']
dashstyle = ['-', '--', '-.', ':', ':']
c=[]
m=[]

In [ ]:
import topology


alpha = alphas[0]
infty_value = 100
#D-SGD
def plot(plot_accuracy=False):
    fig, axs = plt.subplots(1, len(attacks), figsize=(20, 5), sharex=False, sharey=True)
    for attack_id, attack in enumerate(attacks):
        
        for gar_id, gar in enumerate(gars): #+['dsgd']):
            X=[]
            Y=[]
            Yerr=[]
            Ymin=[]
            Ymax=[]
            for top_hyper_id, top_hyper in  enumerate(topology_hypers):
                #Gar generic
                if gar == 'dsgd':
                    name_experiment = f"{dataset}-average_sparse-h_{params['nb-honests']}-{params["topology-name"]}_{top_hyper}-m_{momentum}-alpha_{alpha}_dsgd"
                else:
                    name_experiment = f"{dataset}-{attack}-{gar}-f_{f}-h_{params['nb-honests']}-{params["topology-name"]}_{top_hyper}-m_{momentum}-alpha_{alpha}"
                
                for seed in seeds:  
                    path = result_directory + name_experiment + "-" +str(seed) + "/eval"
                    try: 
                        df_seed = pd.read_csv(path, delimiter='\t')
                    except:
                        # print(f"file not found:{path}")
                        continue

                    net = topology.create_graph("Erdos_Renyi", size=params['nb-honests'], hyper=top_hyper, seed=seed, byz=0, weights_method='unitary')

                    X.append(net.algebraic_connectivity)
                    out = df_seed[['Cross-accuracy', 'Average loss']].values[-1,:]
                    if plot_accuracy:
                        Y.append(out[0]) ## Plot the train loss
                    else:
                        Y.append(out[1])

            Y=np.array(Y)
            if not plot_accuracy:
                nan_indices = (np.isnan(Y) | (Y>100))
                Y = np.where(nan_indices, infty_value, Y)
            
            X=np.array(X)
            indices_sort = np.argsort(X)
            X=X[indices_sort]
            Y=Y[indices_sort]

            if len(X)==0:
                continue
            axs[attack_id].plot(X,Y, color=colors[gar_id], marker=markers[gar_id], label=algorithms_name[gar], linestyle=dashstyle[gar_id])
        
        if not plot_accuracy:
            axs[attack_id].axhline(y=infty_value, color="grey", linestyle="dashed", alpha=0.1)

        # Modify xticks: Change indices with NaN values to 'NaN'
        ticks = range(21)
        ticks_labels = []
        for t in ticks:
            if t%5==0:
                ticks_labels.append(t)
            else:
                ticks_labels.append('')
        axs[attack_id].set_xticks(ticks, labels=ticks_labels)
        axs[attack_id].set_title(attack_name[attack])
        axs[attack_id].set_xlabel(r"$\mu_2$")
        axs[attack_id].set_xlim(left=0.)

        #axs[attack_id].set_xticks(ticks=byzcounts)
    if plot_accuracy:
        ylabel = "accuracy"
        axs[0].set_ylim(0, 1)

    else:
        ylabel = "train loss"
        axs[0].set_yscale("log")
        ytick_values = [ 1e-1, 1e0, 1e1, 1e2]
        ytick_labels = [ '1e-1', '1e0', '1e1', 'inf']
        axs[0].set_yticks(ytick_values, ytick_labels)

    axs[0].set_ylabel(ylabel,rotation=90,size="large")

    handles, labels = axs[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.52, 0.05), ncol=5, labelspacing=0.1, handletextpad=0.1, borderaxespad=0)
    fig.tight_layout(w_pad=0.2)
    
    if plot_accuracy:
        name_plot = f"accuracy_{dataset}-f_{f}-n_{params['nb-honests']}-{params["topology-name"]}-alpha_{alpha}"
    else:
        name_plot = f"loss_{dataset}-f_{f}-n_{params['nb-honests']}-{params["topology-name"]}-alpha_{alpha}"
    
    fig.savefig(plot_directory + name_plot+'.pdf',bbox_inches='tight')
    plt.show()


plot(False)
plot(True)